In [4]:
from langchain.chains import APIChain
from langchain.llms.openai import OpenAI
from langchain.requests import RequestsWrapper
from langchain_community.agent_toolkits.openapi import planner
from langchain_community.agent_toolkits.openapi.spec import reduce_openapi_spec

In [8]:
import requests
import json

res = requests.get('https://kartkatalog.geonorge.no/api/search?text=berggrunn')
res.json()

{'NumFound': 15,
 'Limit': 10,
 'Offset': 1,
 'Results': [{'Uuid': '43dc3e09-3bac-49b1-9b34-62358482f58c',
   'Title': 'Berggrunn N1350',
   'Abstract': 'Berggrunndataene viser utbredelsen på nasjonal målestokk (1:1 350 000) av de forskjellige bergarter på jordens overflate, samt strukturer i bergartene og opplysninger om bergartenes aldersforhold. Dataene er hovedsakelig basert på landsdekkende berggrunnsgeologiske kartdata i målestokk 1:250.000. Kunnskap om berggrunnen spiller en viktig rolle i leting etter og vurdering av ulike typer råmaterialer og naturressurser, samt innen vannforsyning, avfallslagring, areal- og miljøplanlegging, anleggsvirksomhet, alternativ energi og vurdering av naturfarer som jordskjelv og skred. Berggrunnsdata i målestokk 1:1 350.000 er beregnet for å gi en nasjonal oversikt.',
   'Type': 'dataset',
   'TypeTranslated': 'Datasett',
   'Theme': 'Geologi',
   'Organization': 'Norges geologiske undersøkelse',
   'OrganizationLogo': 'https://register.geonorge.n

In [5]:
import requests

# openapi_spec_url = "https://alenos-tester001.azurewebsites.net/openapi?f=json"
openapi_spec_url = "https://ws.geonorge.no/adresser/v1/openapi.json"

def get_json_api_endpoint(url):
    response = requests.get(url)
    response.raise_for_status()  
    return response.json()

def remove_external_refs(spec):
    """
    Recursively search for $ref keys in the OpenAPI spec and remove those that do not start with '#'.
    """
    if isinstance(spec, dict):
        for key, value in list(spec.items()):
            if key == "$ref" and not value.startswith("#"):
                del spec[key]
            else:
                remove_external_refs(value)
    elif isinstance(spec, list):
        for item in spec:
            remove_external_refs(item)

raw_alenos_api_spec = get_json_api_endpoint(openapi_spec_url)
remove_external_refs(raw_alenos_api_spec)
raw_alenos_api_spec

{'info': {'description': 'API for oppslag av adresser i matrikkelen.\n            Det er ikke nødvendig med innlogging/autorisasjon for å bruke API-et.\n            Større funksjonalitetsødeleggende endringer i API-et vil bli annonsert minst 3 måneder i forveien på https://geonorge.no/aktuelt/varsler/Tjenestevarsler/\n            API-et returnerer kun de første 10 000 resultatene. Hvis man ønsker å hente ned større datasett så anbefales det å laste ned filene som er tilgjengelige fra https://geonorge.no .\n        ',
  'title': 'Åpent adresse-API fra Kartverket',
  'version': '1.2.0'},
 'consumes': ['application/json'],
 'produces': ['application/json'],
 'host': 'ws.geonorge.no',
 'basePath': '/adresser/v1',
 'schemes': ['https'],
 'paths': {'/sok': {'get': {'summary': 'Standard søk.',
    'description': 'Søk etter adresser. Minst ett søkeparameter må benyttes. For generelle søk så anbefales det å benytte søkeparameteret "sok", og så eventuelt snevre inn resultatet ved å bruke de andr

In [6]:
alenos_api_spec = reduce_openapi_spec(raw_alenos_api_spec, dereference=False)
alenos_api_spec.endpoints

KeyError: 'servers'

In [ ]:
endpoints = [
    (route, operation)
    for route, operations in raw_alenos_api_spec["paths"].items()
    for operation in operations
    if operation in ["get", "post"]
]
len(endpoints)

20

In [ ]:
import json
import tiktoken

enc = tiktoken.encoding_for_model('gpt-4')

def count_tokens(s):
    return len(enc.encode(s))

print(count_tokens(json.dumps(raw_alenos_api_spec)))
print(count_tokens(json.dumps(alenos_api_spec.endpoints)))

14577
1417
